In [47]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
import re
import copy
import json
from datetime import timedelta
%matplotlib inline

In [6]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

from IPython.display import Audio
sound_file = 'beep.wav'
# pd.set_option('display.max_colwidth', -1)

## Importing data

Importing a sample of the dataset:

In [7]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

Importing the whole dataset

In [8]:
columns_header = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId',
                  'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl',
                 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount',
                 'userLocation']

filename = os.path.join('data','twex.tsv') # 'sample.tsv')
pr('Starting to read file... (3 min)')
# tw = pd.read_csv(filename, sep='\t', encoding='utf-8', escapechar='\\', names=columns_header,
#                       quoting=csv.QUOTE_NONE, na_values='N', header=None)

pr('File is loaded!')
# Audio(url=sound_file, autoplay=True)

10:57:19 Starting to read file... (3 min)
10:57:19 File is loaded!


In [9]:
tw.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,9514097914,1.7341e+07,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197,NaN,NaN,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586,508.0,9016.0,"Bern, Switzerland"
1,9514846412,7.19828e+06,2010-02-23 06:22:40,Still the best coffee in town — at La Stanza h...,8.53781,47.3678,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Nico Luchsinger,halbluchs,1820,703.0,4687.0,"Zurich, Switzerland"


## Extrating hashtags

In [10]:
def extract_hashtags(text):
    ht_list = re.findall(r"#(\w+)", text)
    non_empty_hts = list(filter((lambda ht: ht != []), ht_list))
    lowerCharList = [ht.lower() for ht in non_empty_hts]
    return lowerCharList

In [11]:
pr('Making hashtags... (2 min)')
tw['hashtag'] = np.nan
tw.hashtag = tw.text.apply(lambda x: extract_hashtags(str(x)))
twh = tw.ix[tw.hashtag.apply(lambda x: len(x) != 0)]
pr('Done.')

10:57:24 Making hashtags... (2 min)
10:57:24 Done.


In [12]:
twh.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,...,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
8,9519737890,1.46579e+07,2010-02-23 09:59:41,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,...,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
10,9521789689,9.96202e+06,2010-02-23 11:28:27,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,...,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


## Cleaning data and making date index

In [13]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

The data have been reduced from 19719 tweets to 19719 tweets.


In [14]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

10:57:26 Removing bad dates...
10:57:26 Finished.


In [15]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

10:57:27 Starting to examine dates...
10:57:27 There are 0 dates that cannot be transformed.


In [16]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

10:57:27 Starting copy...
10:57:27 Converting to datetime...
10:57:27 Setting up new indices...
10:57:27 Deleting old "createdAt" column...
10:57:27 Done!


,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,1.46579e+07,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,NaN,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
2010-02-23 11:28:27,9521789689,9.96202e+06,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


In [17]:
tw5['hashtag'][:6]

createdAt
2010-02-23 09:59:41               [fb]
2010-02-23 11:28:27              [yam]
2010-02-23 17:47:11          [24, vfb]
2010-02-23 18:19:03    [iphoneography]
2010-02-23 18:31:46     [partnermonth]
2010-02-24 06:09:23      [insider, fb]
Name: hashtag, dtype: object

## Let's put one hashtag per row

We will make a dataframe with one row = one hashtag. This will be done by going through the dataframe, and making in parallel a list of rows (with 1 hashtag per row) that needs to be added to the old dataframe.

In [18]:
addedHashtagsRowsList = []
def multiplyHashtagRows(row, columns):
    '''
    Examine each row. If there are multiple hashtags, it will return the first one.
    (so the first one will replace the list of hashtags in the df). Then for all the next ones,
    it will make a copy of the row in the addedHashtagsRowsList, (in a dictionary format).
    So this dictionary can in the end be transformed in a DF and added to the original DF.
    (The speed is increased a lot by doing it this way!)
    '''
    htList = row.hashtag
    if len(htList) > 1:
        ## Making the dictionary
        addedHashtag = {}
        addedHashtag['createdAt'] = row.name #the df index
        for col in columns:
            addedHashtag[col] = row[col]
        ## Copying the dict for each hashtag
        i = 1
        while i < len(htList) :
            deepCopy = copy.deepcopy(addedHashtag)
            deepCopy['hashtag'] = htList[i]
            addedHashtagsRowsList.append(deepCopy)
            i+=1
    return htList[0] # return the first hashtag

In [19]:
addedHashtagsRowsList = []
tw5_1 = tw5.copy()
pr('Multiplying the hashtag rows... (around 10 min)')
tw5_1['hashtag'] = tw5.apply(multiplyHashtagRows, args=[tw5.columns,], axis=1)
pr('Finished! {} rows will be added to the dataframe!'.format(len(addedHashtagsRowsList)))

10:57:28 Multiplying the hashtag rows... (around 10 min)
10:57:33 Finished! 8887 rows will be added to the dataframe!


In [20]:
pr('Starting to make the new dataframe with additionnal rows..')
addedHashtagsDf = pd.DataFrame(addedHashtagsRowsList)
addedHashtagsDf.set_index(['createdAt'], inplace=True)
pr('Starting to append the two df... Old df size = {}'.format(len(tw5_1)))
tw6 = tw5_1.append(addedHashtagsDf)
pr('Done! New df size = {}'.format(len(tw6)))

10:57:33 Starting to make the new dataframe with additionnal rows..
10:57:33 Starting to append the two df... Old df size = 19719
10:57:33 Done! New df size = 28606


In [21]:
print('Example hahshtag:')
tw6[tw6['hashtag'] == addedHashtagsRowsList[0]['hashtag']].head(3)

Example hahshtag:


,followersCount,friendsCount,hashtag,id,inReplyTo,latitude,longitude,placeId,placeLatitude,placeLongitude,screenName,source,sourceName,sourceUrl,statusesCount,text,truncated,userId,userLocation,userName
createdAt,,,,,,,,,,,,,,,,,,,,
2011-07-24 16:03:13,1121,763.0,vfb,95162125584039936,9.501397e+16,NaN,NaN,e401fb8eb4e7595a,47.5356,9.14004,eLd0raDo,14,Tweetbot for iOS,http://tapbots.com/tweetbot,6735.0,"@mikstweed Wie lange ist es her, dass der #vfb...",NaN,921241,"Kreuzlingen, TG, Switzerland",Markus Tressl
2010-02-23 17:47:11,1121,763.0,vfb,9535390586,NaN,47.6463,9.16570,NaN,NaN,NaN,eLd0raDo,550,Gowalla,http://gowalla.com/,6735.0,"So, Feierabend. Jetzt #24 und später #VfB. — a...",NaN,921241,"Kreuzlingen, TG, Switzerland",Markus Tressl
2010-12-01 17:25:56,1317,830.0,vfb,10021795398684672,NaN,46.9499,7.47071,e38a1a641d02f8db,46.9543,7.39491,chm,1,Twitter for iPhone,http://twitter.com/#!/download/iphone,13552.0,"So, und jetzt Flachmann suchen und dann ab ins...",NaN,120433,"Bern, Switzerland",chm


In [22]:
tw6.hashtag.head()

createdAt
2010-02-23 09:59:41               fb
2010-02-23 11:28:27              yam
2010-02-23 17:47:11               24
2010-02-23 18:19:03    iphoneography
2010-02-23 18:31:46     partnermonth
Name: hashtag, dtype: object

## Grouping per hashtags per day

In [23]:
tw6.dropna(subset=['longitude'], inplace=True)
tw6.dropna(subset=['latitude'], inplace=True)

In [24]:
tw6.latitude = tw6.latitude.apply(float)
tw6.longitude = tw6.longitude.apply(float)

In [25]:
delimiter = '_$$$_'
str_join = lambda x: delimiter.join(x)

Function that applies to a dataframe will group each row by day and aggregate all its content:

In [26]:
def aggDate(df):
    groupedDf = df.groupby(df.index.map(lambda x: x.date)).agg({'text' : str_join,
                                                          'longitude' : np.median,
                                                          'latitude' : np.median,
                                                          'hashtag' : lambda x: x.iloc[0], ## the first occurence
                                                          'numberOfTweets' : 'count'})
    return groupedDf

In [27]:
pr('Making column number of tweets')
tw6['numberOfTweets'] = 1
pr('Starting group by hastag...')
gp = tw6.groupby('hashtag')
pr('Starting to put hashtag in dictionary... (around 50 min)')

count = 0
lengp = len(gp)
printingValue = int(lengp / 10)
dictionary = {}
for hashtag, df in gp:
    dictionary[hashtag] = aggDate(df)
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/lengp*100))
pr('Finished operations! Dictionary with {} different hashtags.'.format(len(dictionary)))

10:57:37 Making column number of tweets
10:57:37 Starting group by hastag...
10:57:37 Starting to put hashtag in dictionary... (around 50 min)
10:57:44 10%
10:57:52 20%
10:57:59 30%
10:58:05 40%
10:58:11 50%
10:58:18 60%
10:58:24 70%
10:58:30 80%
10:58:37 90%
10:58:43 100%
10:58:43 Finished operations! Dictionary with 9410 different hashtags.


In [29]:
# aggDate(gp.get_group('twibisg')).head(3)
# dictionary['twibisg'].head(3)
print('Dictionary with hashtags dataframes:')
dictionary[list(dictionary.keys())[5]].head(4)

Dictionary with hashtags dataframes:


,hashtag,text,longitude,numberOfTweets,latitude
2010-03-24,passtimes,It's amazing how relaxing it is to watch falli...,6.15248,1,46.2043


## Starting event detection

Parameters that define events:

In [69]:
## Parameters of an event:
MIN_TOT_NB_TWEETS = 20 ## The hashtag must have happened at least this number of times to be considered.
MIN_NB_DAYS_WITH_HASHTAGS = 3 ## The hashtags must appear at least this number of different days to be considered.
MIN_NB_TWEETS_DURING_EVENT = 5 ## To be considered an event, the hashtag must happen at least this nb of times during the day.
THRESHOLD_ANOMALY_FACTOR = 3 ## The occurence of a hashtag during a single day must be above the mean by this FACTOR
                             ## multiplied by the std to be considered as an anomaly.
MAX_DURATION_OF_EVENT = timedelta(days=30) ## The maximum number of days we consider an event can happen
MIN_DURATION_BEFORE_NEW_EVENT = timedelta(days=304) ## (= 10 months) The min time that should pass before an event can happen
                                                    ## again and still be considered as event (ie. Christmas is an event
                                                    ## each year)

Helper functions to detect recurrent events that should be removed:

In [63]:
def isSpecificEventListIllegal(detectedEventDateList):
    '''
    Return true if the list of dates contain illegal tupples of events, so if the event is recurrent
    which would mean it is not a real event.
    '''
    def datesAreIllegal(date1, date2, date3):
        '''
        Return true if the 3 dates are not to be considered as regular events.
        '''
        ## Return if the difference is too small to be considered as 2 different events
        def diffIsSmall(timeDiff):  
            return timeDiff < MAX_DURATION_OF_EVENT

        ## Return true if the difference is not big enough to be an annual event.
        def isDiffSuspect(timeDiff):
            return timeDiff < MIN_DURATION_BEFORE_NEW_EVENT   

        diff1 = abs(date1 - date2)
        diff2 = abs(date2 - date3)
        diff3 = abs(date3 - date1)

        ## The difference is too small, it must be the same event
        if diffIsSmall(diff1) or diffIsSmall(diff2) or diffIsSmall(diff3):
            return False

        ## If there are at least 2 out of 3 suspect difference, then the dates are illegal
        if isDiffSuspect(diff1):
            return isDiffSuspect(diff2) or isDiffSuspect(diff3)
        else:
            return isDiffSuspect(diff2) and isDiffSuspect(diff3)
    
    ## MAIN FUNCTION : ##
    # Go through the list of events and try all "triples" to see if there is any illegal triples. This is a quickly done
    # code to do that. Code complexity bellow is in O(k^3), with k being the size of the list. We will apply this function
    # to n list so we will have an overall complexity in O(n*k^3). We can consider however that each list will
    # be small so k can be considered as constant and therefore the overall complexity will be in O(n).
    for i in range(len(detectedEventDateList) - 2):
        for j in range(i, len(detectedEventDateList) - 1):
            for k in range(j, len(detectedEventDateList)):
                if datesAreIllegal(detectedEventDateList[i], detectedEventDateList[j], detectedEventDateList[k]):
                    return True
    return False

In [64]:
pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
nbOfEventDetected = 0
count = 0
printingValue = int(len(dictionary) / 10)
for [h,df] in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df['event'] = False
    if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
        if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
            threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
            df['event'] = df.numberOfTweets.apply(lambda x: x > threshold and x > MIN_NB_TWEETS_DURING_EVENT)
            
            ## Remove recurrent events:
            detectedEventDf = df[df['event']]
            if len(detectedEventDf) > 3 and isSpecificEventListIllegal(detectedEventDf.index):
                df['event'] = False
            nbOfEventDetected += len(df[df['event']])
pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

11:23:00 Starting to compute 9410 dict items to detect event. (4 min)
11:23:00 10%
11:23:00 20%
11:23:00 30%
11:23:01 40%
11:23:01 50%
11:23:01 60%
11:23:01 70%
11:23:02 80%
11:23:02 90%
11:23:02 100%
11:23:02 Finished! Number of events detected = 14


## Making single event dataframe

In [65]:
eventRowsList = []
def applyToMakeEventDf(row):
    if row.event:
        rowToAdd = {'date': row.name, 'hashtag': row.hashtag, 'text': row.text,
                    'longitude': row.longitude, 'latitude':row.latitude, 'numberOfTweets': row.numberOfTweets, }
        eventRowsList.append(rowToAdd)

In [66]:
eventRowsList = []
count = 0
printingValue = int(len(dictionary) / 10)

pr('Starting to make event df with {} dataframes. (around 6 min)'.format(len(dictionary)))
for h, df in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df.apply(applyToMakeEventDf, axis=1)

pr('Making new dataframe.')
new_events = pd.DataFrame(eventRowsList)
new_events.set_index(['date'], inplace=True)
pr('Finished! Dataframe with {} rows'.format(len(new_events)))

11:23:05 Starting to make event df with 9410 dataframes. (around 6 min)
11:23:05 10%
11:23:06 20%
11:23:07 30%
11:23:07 40%
11:23:08 50%
11:23:09 60%
11:23:09 70%
11:23:10 80%
11:23:11 90%
11:23:11 100%
11:23:11 Making new dataframe.
11:23:11 Finished! Dataframe with 14 rows


In [67]:
print('Old events df:')
oldEvents

Old events df:


,hashtag,latitude,longitude,numberOfTweets,text
date,,,,,
2010-06-16,worldcup,47.21025,8.409725,12,With Swiss colleagues watching Switzerland vs ...
2011-08-02,android,47.50000,8.750000,6,@AniseSmith oh no! But that means I have to b...
2011-10-06,luzern,47.05095,8.308910,6,Landscape with the Lac des quatre cantons. #lu...
2011-09-07,smgzh,47.36490,8.546870,13,"Läck! Ist das früh heute... #smgzh_$$$_#smgzh,..."
2010-06-16,sui,47.37880,8.538030,25,"#sui wonderful_$$$_en plus du goal, j'aurai mi..."
2010-07-30,iphone4,47.37540,8.538800,10,"Ok, here's the queue. 05:56 CET #iPhone4 #aszh..."
2010-12-08,switzerland,47.36380,8.534930,9,India's cell-phone marketing: 545 million hand...
2011-06-18,zvilla,47.22570,8.818970,29,Die Regenblache ist gespannt. Auf der #zVilla ...
2011-08-09,twibisg,47.48410,9.549050,6,genau die gleiche frage wollte ich auch an @va...


In [68]:
print('Events dataframe:')
new_events

Events dataframe:


,hashtag,latitude,longitude,numberOfTweets,text
date,,,,,
2010-06-16,worldcup,47.21025,8.409725,12,With Swiss colleagues watching Switzerland vs ...
2011-08-02,android,47.50000,8.750000,6,@AniseSmith oh no! But that means I have to b...
2011-10-06,luzern,47.05095,8.308910,6,Landscape with the Lac des quatre cantons. #lu...
2010-07-30,iphone4,47.37540,8.538800,10,"Ok, here's the queue. 05:56 CET #iPhone4 #aszh..."
2010-12-08,switzerland,47.36380,8.534930,9,India's cell-phone marketing: 545 million hand...
2011-06-18,zvilla,47.22570,8.818970,29,Die Regenblache ist gespannt. Auf der #zVilla ...
2011-03-25,ipad2,47.37510,8.538960,41,@greezer der @ThBenkoe hat's schon #iPad2 ht...
2010-12-03,ff,47.58760,9.624350,6,Huiiiiiii ich bin von @jetztlaecheln und @seli...
2011-06-10,ff,47.38020,8.518810,7,Hoy mi #ff va para @Palermo41 @Brais_sete y @t...


In [35]:
print('Linked dataframe of all days:')
dictionary[new_events.iloc[0].hashtag].head(10)

Linked dataframe of all days:


,hashtag,text,longitude,numberOfTweets,latitude,event
2010-06-10,worldcup,Les italiens sont prêts pour la coupe du monde...,7.380340,1,46.23410,False
2010-06-11,worldcup,#worldcup C'est parti pour un mois de foot \o/...,6.645310,3,46.51730,False
2010-06-12,worldcup,I would blame bp for that result! #worldcup #e...,8.527640,1,47.39250,False
2010-06-13,worldcup,"""Vokuhila oder wie die Tröten heissen."" #world...",9.159270,2,47.35055,False
2010-06-14,worldcup,Mondiali 2010: si comincia. http://twitpic.com...,9.664400,1,45.83860,False
2010-06-16,worldcup,With Swiss colleagues watching Switzerland vs ...,8.409725,12,47.21025,True
2010-06-17,worldcup,Kicked out of the house by Knitting Club so fo...,7.436690,1,46.94690,False
2010-06-18,worldcup,"Könnt ihr nicht konkreter tweeten, damit ich a...",7.951700,1,47.55150,False
2010-06-24,worldcup,I'm so happy Italy and France are out of the #...,8.540860,1,47.36430,False
2010-06-26,worldcup,Meine #worldcup App zeigt morgen Nachmittag pl...,7.433060,1,46.94290,False


## Exporting data

In [55]:
e_df = new_events.copy()
e_df['date'] = e_df.index
e_df.index = [i for i in range (len(e_df))]
e_df.head(1)

,hashtag,latitude,longitude,numberOfTweets,text,date
0,worldcup,47.21025,8.409725,12,With Swiss colleagues watching Switzerland vs ...,2010-06-16


We are going to generate the right datetimes for the jsons:

In [56]:
epoch_dt = datetime(1970, 1, 1)
def to_utc(date):
    d_dt = datetime.combine(date, datetime.min.time())
    return int((d_dt - epoch_dt).total_seconds()*1000)

In [57]:
def convert_to_unix_time(record):
    datetime_index = pd.DatetimeIndex([datetime(record['year'], record['month'], 1)])
    unix_time_index = datetime_index.astype(np.int64) // 10**6
    return unix_time_index[0]

In [58]:
pr('Converting dates...')
e_df['year'] = e_df['date'].apply(lambda x: x.year)
e_df['month'] = e_df['date'].apply(lambda x: x.month)
e_df['utc_date'] = e_df['date'].apply(lambda x: to_utc(x))
e_df['unix_time'] = e_df.apply(convert_to_unix_time, axis=1)
pr('Done.')
e_df.head(1)

11:22:33 Converting dates...
11:22:33 Done.


,hashtag,latitude,longitude,numberOfTweets,text,date,year,month,utc_date,unix_time
0,worldcup,47.21025,8.409725,12,With Swiss colleagues watching Switzerland vs ...,2010-06-16,2010,6,1276646400000,1275350400000


Grouping by months and generating the dictionary for the json.

In [59]:
e_gb_month = e_df.groupby(e_df.unix_time)

In [60]:
pr('Making event list...')
months = []
for month, df in e_gb_month:
    days = []
    for i in range (len(df)):
        ht = df.iloc[i]['hashtag']
        lat = df.iloc[i]['latitude']
        lon = df.iloc[i]['longitude']
        t_num = df.iloc[i]['numberOfTweets']
        tweets = df.iloc[i]['text'].split(delimiter)
        date = df.iloc[i]['utc_date']
        
        data_unit = { 'name': ht
                    , 'latitude' : lat
                    , 'longitude' : lon
                    , 'tweets' : tweets
                    , 'number_of_tweets' : str(t_num)
                    , 'date' : int(date)}
        days.append(data_unit)
    
    curr_month = {'date': int(month), 'data' : days}
    months.append(curr_month)

final_events = {'events' : months}
pr('Done.')

11:22:34 Making event list...
11:22:34 Done.


In [61]:
exportFilename = 'export_twitter_events_' + datetime.now().strftime("%Y-%m-%d_%Hh%Mmin%S") + '.json'
exportPath =  os.path.join('data', exportFilename)

pr('Exporting to json...')
with open(exportPath, 'w') as f:
     json.dump(final_events, f)
pr('Export done. File "{}" has been created.'.format(exportFilename))

11:22:35 Exporting to json...
11:22:35 Export done. File "export_twitter_events_2017-01-30_11h22min35.json" has been created.
